In [0]:
!pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/pfm/')

In [4]:
import config
import noise_generator


noise file noisy_label2_kv.txt generated with noise: 0.9



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [0]:
import random
import numpy as np
random.seed( 30 )
r = 0.5

In [0]:
# noise_generator.generate_noise(config.noise_rate,  symmetric = True, datadir = config.data_dir, train_file_name = "noisy_label_kv2.txt")

In [0]:
import pandas as pd
df_data_mapping = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/pfm/data/image_set_mapping.csv')

df_clean_labels = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/pfm/data/clean_label_kv.txt', sep = " ", header = None)
df_clean_labels.columns = ['directory', "label"]

#get a full dataframe containing sets and classes
df = df_data_mapping.merge(df_clean_labels, how = 'left', right_on = "directory", left_on = "directory",  left_index = False, right_index = False)

In [72]:
df

,Unnamed: 0,directory,class_number,set,label
0,0,images/train/airplane/0001.png,0,train,0
1,0,images/train/airplane/0002.png,0,train,0
2,0,images/train/airplane/0003.png,0,train,0
3,0,images/train/airplane/0004.png,0,train,0
4,0,images/train/airplane/0005.png,0,train,0
...,...,...,...,...,...
59995,0,images/train/truck/4996.png,9,validation,9
59996,0,images/train/truck/4997.png,9,validation,9
59997,0,images/train/truck/4998.png,9,validation,9
59998,0,images/train/truck/4999.png,9,validation,9


In [0]:
val_df  = df[df.set == "validation"][["directory","class_number","set","label"]].copy()
test_df = df[df.set == "test"][["directory","class_number","set","label"]].copy()

In [0]:
#generate a list of random labels
set_classes = set(df.label)
randomlist = []
for i in range(0,val_df.shape[0]):
    n = random.randint(min(set_classes),max(set_classes))
    randomlist.append(n)
val_df["random_class"] = randomlist

In [0]:
#column of probabilities (uniform)
val_df["p"] = np.random.uniform(0,1,val_df.shape[0])

In [0]:
#use the random list to generate noise in effective percentage r
N_CLASSES = len(set_classes)
NOISE_PERCENTAGE = r*N_CLASSES/(N_CLASSES-1)
noisy_class = []
for index, row in val_df.iterrows():
    if row["p"]<= NOISE_PERCENTAGE:
        noisy_class.append( row["random_class"])
    else:
        noisy_class.append(row["class_number"])

val_df["noisy_class"] = noisy_class

In [77]:
val_df.head(10)

,directory,class_number,set,label,random_class,p,noisy_class
55000,images/train/airplane/4501.png,0,validation,0,8,0.786917,0
55001,images/train/airplane/4502.png,0,validation,0,4,0.502619,4
55002,images/train/airplane/4503.png,0,validation,0,9,0.189548,9
55003,images/train/airplane/4504.png,0,validation,0,0,0.967676,0
55004,images/train/airplane/4505.png,0,validation,0,9,0.616991,0
55005,images/train/airplane/4506.png,0,validation,0,3,0.411004,3
55006,images/train/airplane/4507.png,0,validation,0,4,0.374118,4
55007,images/train/airplane/4508.png,0,validation,0,0,0.992752,0
55008,images/train/airplane/4509.png,0,validation,0,6,0.467936,6
55009,images/train/airplane/4510.png,0,validation,0,6,0.817621,0


In [0]:
val_df["class_number"] = val_df["noisy_class"]
export = val_df[["directory","class_number"]].append(test_df)[["directory", "class_number"]]

In [83]:
export

,directory,class_number
55000,images/train/airplane/4501.png,0
55001,images/train/airplane/4502.png,4
55002,images/train/airplane/4503.png,9
55003,images/train/airplane/4504.png,0
55004,images/train/airplane/4505.png,0
...,...,...
54995,images/test/truck/0996.png,9
54996,images/test/truck/0997.png,9
54997,images/test/truck/0998.png,9
54998,images/test/truck/0999.png,9
